In [2]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import re

awdd;flkjasdflkajsdf          a;'jc xk ,

In [3]:
ram_ppp = pd.read_csv('data/raw/ppp/memory.csv')
ram_local = pd.read_csv('data/filtered/prices_ram.csv')

filter ppp ram


In [ ]:
def valid_ram(modules):
    sticks, per_stick = map(int, modules.split(','))
    total = sticks * per_stick
    return total >= 8 and not (sticks == 2 and per_stick == 4)

ram_ppp = ram_ppp[ram_ppp['modules'].apply(valid_ram)]


ram_ppp.head()

In [ ]:
ram_ppp[['ddr_gen', 'mhz']] = ram_ppp['speed'].str.split(',', expand=True)

ram_ppp['ddr_gen'] = ram_ppp['ddr_gen'].astype(int)
ram_ppp['mhz'] = ram_ppp['mhz'].astype(int)

ram_ppp = ram_ppp.drop(columns=['speed'])



In [ ]:
ram_ppp = ram_ppp[ram_ppp['ddr_gen'] >= 4]

ram_ppp.to_csv('data/filtered/ppp_filtered_ram.csv', index=False)

In [ ]:
def parse_local_ram(df):
    # Create new columns for parsed values
    df['total_capacity_gb'] = None
    df['modules'] = None
    df['ddr_gen'] = None
    df['mhz'] = None
    df['cl'] = None
    df['voltage'] = None
    
    for i, row in df.iterrows():
        name = row['item_name'].lower()
        
        # extract total capacity (64gb)
        cap_match = re.search(r'(\d+)\s?gb', name)
        total_capacity = int(cap_match.group(1)) if cap_match else None
        
        # check if dual/kit is mentioned
        if "(dual" in name or "2x" in name or "2 x" in name:
            modules = f"2,{total_capacity // 2}" if total_capacity else None
        else:
            modules = f"1,{total_capacity}" if total_capacity else None

        # generation (ddr4 or ddr5)
        if "ddr4" in name:
            ddr_gen = 4
        elif "ddr5" in name:
            ddr_gen = 5
        else:
            ddr_gen = None
        
        # speed (MHz)
        mhz_match = re.search(r'ddr[45]\s*(\d{3,5})', name)
        mhz = int(mhz_match.group(1)) if mhz_match else None
        
        # --- CAS Latency (CL)
        cl_match = re.search(r'cl(\d+)', name)
        cl = int(cl_match.group(1)) if cl_match else None
        
        # --- Voltage (e.g., 1.35v)
        volt_match = re.search(r'(\d\.\d+)\s?v', name)
        voltage = float(volt_match.group(1)) if volt_match else None
        
        # Assign extracted values
        df.at[i, 'total_capacity_gb'] = total_capacity
        df.at[i, 'modules'] = modules
        df.at[i, 'ddr_gen'] = ddr_gen
        df.at[i, 'mhz'] = mhz
        df.at[i, 'cl'] = cl
        df.at[i, 'voltage'] = voltage

    return df


In [ ]:
ram_local = parse_local_ram(ram_local)
ram_local.head()

ram_local.to_csv('data/filtered/prices_ram.csv', index=False)

In [ ]:
ram_local = ram_local[['total_capacity_gb', 'ddr_gen', 'mhz', 'item_price_in_php', 'voltage']]

merged_ram = pd.merge(
    ram_ppp,
    ram_local,
    left_on=['ddr_gen', 'mhz', 'total_capacity'],
    right_on=['ddr_gen', 'mhz', 'total_capacity_gb'],
    how='inner'
)


In [ ]:
local_ram_grouped = (
    ram_local
    .groupby(['ddr_gen', 'mhz', 'total_capacity_gb'], as_index=False)
    .agg({
        'item_price_in_php': 'min',  
        'voltage': 'first'           
    })
)

merged_ram = pd.merge(
    ram_ppp,
    local_ram_grouped,
    left_on=['ddr_gen', 'mhz', 'total_capacity'],
    right_on=['ddr_gen', 'mhz', 'total_capacity_gb'],
    how='inner'
).drop(columns=['total_capacity_gb'])

merged_ram.to_csv('data/processed/ram_merged.csv', index=False)


In [ ]:

def clean_item_name(name):
    return re.sub(r'^\s*\d+gb\s*(?:\([^)]+\))?\s*ddr\d\s*\d+\s*', '', name, flags=re.IGNORECASE)

def strip_suffix(name):
    return name.split(',')[0].strip()



ram_local['clean_name'] = ram_local['item_name'].apply(clean_item_name).apply(strip_suffix)



ram_local.to_csv('data/processed/ram_merged.csv')


KeyError: 'item_name'

In [13]:

ram_local = ram_local.drop(columns=['category_name'])
ram_local.to_csv('data/processed/ram_merged.csv')


In [22]:

ram_local['item_price_in_php'] = ram_local['item_price_in_php'].astype(float)
ram_local['stocks'] = ram_local['stocks'].astype(int)
ram_local['total_capacity_gb'] = ram_local['total_capacity_gb'].astype(int)
ram_local['num_modules'] = ram_local['num_modules'].astype(int)
ram_local['capacity_per_module_gb'] = ram_local['capacity_per_module_gb'].astype(int)
ram_local['ddr_gen'] = ram_local['ddr_gen'].astype(int)
ram_local['mhz'] = ram_local['mhz'].astype(int)
ram_local['cl'] = ram_local['cl'].astype(float)
ram_local['voltage'] = ram_local['voltage'].astype(float)

ram_local['brand'] = ram_local['clean_name'].str.split().str[0]

proper_order = [
    'brand', 'clean_name', 'item_price_in_php', 'stocks',
    'total_capacity_gb', 'num_modules', 'capacity_per_module_gb',
    'ddr_gen', 'mhz', 'cl', 'voltage'
]
ram_local = ram_local[proper_order]

ram_local = ram_local[~ram_local['clean_name'].str.contains('sodimm', case=False, na=False)]

ram_local.to_csv('data/processed/ram_merged.csv')


In [18]:
print(ram_local.columns.tolist())


['item_price_in_php', 'stocks', 'total_capacity_gb', 'ddr_gen', 'mhz', 'cl', 'voltage', 'clean_name', 'num_modules', 'capacity_per_module_gb']
